## Topological gauge / frequency coupling playground

Core model:

\[
G=\frac{C}{\phi^m} \quad,\quad F_0 = \phi^m\frac{k_B K}{h}
\]

Invariant:

\[
G\,F_0 = C\,\frac{k_B K}{h}
\]

This notebook focuses on **visualizing** how gauge-derived candidate C values intersect targets as m varies (integer ladder).


## Topological gauge / frequency coupling playground

Model:

\[
G=\frac{C}{\phi^m} \quad,\quad F_0 = \phi^m\frac{k_B K}{h}
\]

Invariant:

\[
G\,F_0 = C\,\frac{k_B K}{h}
\]



In [ ]:
from physics_test.model import phi, gauge_G, frequency_F0, coupling_invariant, fit_C_for_target_G
from physics_test import constants

phi()


In [ ]:
# Your example: C=360, m=2
G = gauge_G(360, 2)
inv_alpha = 1 / constants.FINE_STRUCTURE
G, inv_alpha, G - inv_alpha


In [ ]:
# What C would exactly match 1/alpha at m=2?
C_fit = fit_C_for_target_G(inv_alpha, 2)
C_fit


In [ ]:
# Pairing to frequency at some temperature (m is an integer harmonic step)
m: int = 2
K = 300

F0 = frequency_F0(m, K)
G = gauge_G(360, m)

F0, G * F0, coupling_invariant(360, K)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from physics_test.targets import known_targets
from physics_test.gauge_groups import standard_model_gauge_groups, candidate_Cs_from_group

# Convenience
TARGETS = {t.name: t.value for t in known_targets()}

# Gauge-derived C candidates from base=360
Cs = {}
for g in standard_model_gauge_groups():
    for k, v in candidate_Cs_from_group(g, base=360.0).items():
        Cs[f"{g.name}:{k}"] = v

# De-duplicate identical C values but keep one label
C_by_value = {}
for label, c in Cs.items():
    C_by_value.setdefault(float(c), label)

C_list = sorted(C_by_value.keys())
C_labels = [C_by_value[c] for c in C_list]

C_list, C_labels


In [ ]:
def plot_logG_vs_m(
    target_key: str,
    m_min: int = -200,
    m_max: int = 200,
    tol: float = 0.05,
    title: str | None = None,
):
    """Plot log10(G) vs integer m for gauge-derived C candidates, highlighting tolerance hits."""
    target = TARGETS[target_key]
    ms = np.arange(m_min, m_max + 1, dtype=int)

    fig, ax = plt.subplots(figsize=(11, 6))

    # Plot each C curve
    for c, lab in zip(C_list, C_labels):
        Gs = np.array([gauge_G(c, int(m)) for m in ms], dtype=float)
        # avoid log(0)
        logG = np.log10(np.clip(Gs, 1e-300, np.inf))
        ax.plot(ms, logG, lw=1.2, alpha=0.85, label=f"{lab} (C={c:g})")

        # Mark hits within tolerance for this C
        rel_err = (Gs - target) / target
        hit_mask = np.abs(rel_err) <= tol
        if np.any(hit_mask):
            ax.scatter(ms[hit_mask], logG[hit_mask], s=18)

    ax.axhline(np.log10(target), color="black", ls="--", lw=1.5, label=f"target {target_key}")

    ax.set_xlabel("m (integer)")
    ax.set_ylabel("log10(G)")
    ax.set_title(title or f"Gauge-derived C: log10(G) vs m, target={target_key}, tol={tol}")
    ax.grid(True, alpha=0.25)
    ax.legend(ncols=2, fontsize=9)
    plt.show()


# Example plot:
plot_logG_vs_m("sin2thetaW(mZ)", m_min=-50, m_max=50, tol=0.05)


In [ ]:
# Quick comparison plots you can run:
# - Weak coupling (alpha_w)
# - Strong coupling benchmark
# - EM inverse fine-structure

plot_logG_vs_m("alpha_w(mZ)", m_min=-50, m_max=50, tol=0.05)
plot_logG_vs_m("alpha_s(mZ)", m_min=-50, m_max=50, tol=0.05)
plot_logG_vs_m("1/alpha", m_min=-10, m_max=10, tol=0.01)
